# Extraer Informacion de la web con Beautiful Soup

* * * 

#### GRUPO 4 - INTEGRANTES
- CARLOS ALEXANDER CHICAIZA PUEDMAG
- JESSICA VIVIANA LLUMIGUANO CHELA
- EMILIO ISRAEL MAYORGA CAMPOVERDE
- JUAN FRANCISCO VIZUETE VALLEJO

* * * 
### Objetivos de Aprendizaje
1. [Reflexión: ¿Extraer o No Extraer?](#when)
2. [Extraer y Analizar HTML](#extract)
3. [Extraer Información de la Asamblea General de Illinois](#scrape)


<a id='when'></a>

# ¿Extraer o no extraer?

Cuando queremos acceder a datos de la web, primero debemos asegurarnos que el sitio web que nos interesa ofrezca una API web. Plataformas como Twitter, Reddit y el New York Times tienen APIs. Consulta el taller de D-Lab sobre Python Web APIs si quieres aprender cómo usar APIs.

Sin embargo, a menudo no existe una API web. En estos casos, podemos recurrir al "web scraping", que consiste en extraer el HTML subyacente de una página web y obtener directamente la información que necesitamos. Hay varios paquetes en Python que podemos usar para hacer esto. Nos centraremos en dos: Requests y Beautiful Soup.

Nuestro estudio de caso será extraer información sobre los senadores estatales de Illinois, así como la lista de proyectos de ley que cada senador ha patrocinado. Antes de comenzar, echa un vistazo a estos sitios web para ver cómo están estructurados.


## Instalación

Usaremos dos paquetes principales: [Requests](http://docs.python-requests.org/en/latest/user/quickstart/) and [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/). Instala estos paquetes si aún no lo has hecho:


In [5]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


También instalaremos el paquete `lxml`, que ayuda a soportar algunos de los análisis que realiza Beautiful Soup:

In [5]:
%pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Importar las bibliotecas necesarias
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

<a id='extract'></a>

# Extraer y Analizar HTML

Para extraer y analizar HTML de manera exitosa, seguiremos los siguientes 4 pasos:
1. Hacer una solicitud GET
2. Analizar la página con Beautiful Soup
3. Buscar elementos HTML
4. Obtener atributos y texto de estos elementos

## Paso 1: Hacer una solicitud GET para obtener el HTML de una página

Podemos usar la librería Requests para:
1. Hacer una solicitud GET a la página, y
2. Leer el código HTML de la página web.

El proceso de hacer una solicitud y obtener un resultado es similar al flujo de trabajo de una API web. Sin embargo, ahora estamos haciendo una solicitud directamente al sitio web y tendremos que analizar el HTML por nuestra cuenta. Esto es diferente de recibir datos organizados en un formato más sencillo como `JSON` o `XML`.


In [ ]:
# Realizar una solicitud GET
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Leer el contenido de la respuesta del servidor
src = req.text
# Ver algunos resultados
print(src[:1000])

<html lang="en"> 
<!-- Trigger/Open The Modal -->
<div style="position: fixed; z-index: 999; top: 5; left: 600; background-color: navy; display: block">
<button id="myBtn" style="color: white; background-color: navy; display: block">Translate Website</button></div>
<!-- The Modal -->
<div id="myModal" class="modal" style="display: none">
  <!-- Modal content -->
  <div class="modal-content">
      <div class="modal-header"><h3>
    <span class="close">&times;</span></h3></div>    
    <p>The Illinois General Assembly offers the Google Translate service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.</p>
    <p>Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.</p>
    <p>The English language version is always the official and authoritative version of this website.</p>
    <p>NOTE: To return to the original English language version, se


## Paso 2: Analizar la página con Beautiful Soup

Ahora, usamos la función `BeautifulSoup` para analizar la respuesta y convertirla en un árbol HTML. Esto devuelve un objeto (llamado **objeto soup**) que contiene todo el HTML del documento original.

Si encuentras un error relacionado con una librería de análisis, asegúrate de haber instalado el paquete `lxml` para proporcionar a Beautiful Soup las herramientas necesarias para analizar el HTML.

In [ ]:
# Analizar la respuesta en un árbol HTML
soup = BeautifulSoup(src, 'lxml')
# Echar un vistazo
print(soup.prettify()[:1000])

<html lang="en">
 <!-- Trigger/Open The Modal -->
 <body>
  <div style="position: fixed; z-index: 999; top: 5; left: 600; background-color: navy; display: block">
   <button id="myBtn" style="color: white; background-color: navy; display: block">
    Translate Website
   </button>
  </div>
  <!-- The Modal -->
  <div class="modal" id="myModal" style="display: none">
   <!-- Modal content -->
   <div class="modal-content">
    <div class="modal-header">
     <h3>
      <span class="close">
       ×
      </span>
     </h3>
    </div>
    <p>
     The Illinois General Assembly offers the Google Translate service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.
    </p>
    <p>
     Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.
    </p>
    <p>
     The English language version is always the official and authoritative version of this website.
   

La salida se ve bastante similar a la anterior, pero ahora está organizada en un objeto `soup`, lo que nos permite recorrer la página más fácilmente.

## Paso 3: Buscar elementos HTML

Beautiful Soup tiene varias funciones para encontrar componentes útiles en una página. Con Beautiful Soup puedes buscar elementos por:
1. Etiquetas HTML
2. Atributos HTML
3. Selectores CSS

Comencemos buscando **etiquetas HTML**.
La función `find_all` busca en el árbol soup todos los elementos que tienen una etiqueta HTML particular y devuelve todos esos elementos.

¿Que hace el siguiente ejemplo?


In [ ]:
# Encontrar todos los elementos con una etiqueta determinada
a_tags = soup.find_all("a")
print(a_tags[:10])

[<a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a>, <a href="/default.asp"><img alt="Illinois General Assembly" border="0" height="49" src="/images/logo_sm.gif" width="462"/></a>, <a class="mainmenu" href="/">Home</a>, <a class="mainmenu" href="/legislation/" onblur="HM_f_PopDown('elMenu1')" onfocus="HM_f_PopUp('elMenu1',event)" onmouseout="HM_f_PopDown('elMenu1')" onmouseover="HM_f_PopUp('elMenu1',event)">Legislation &amp; Laws</a>, <a class="mainmenu" href="/senate/" onblur="HM_f_PopDown('elMenu3')" onfocus="HM_f_PopUp('elMenu3',event)" onmouseout="HM_f_PopDown('elMenu3')" onmouseover="HM_f_PopUp('elMenu3',event)">Senate</a>, <a class="mainmenu" href="/house/" onblur="HM_f_PopDown('elMenu2')" onfocus="HM_f_PopUp('elMenu2',event)" onmouseout="HM_f_PopDown('elMenu2')" onmouseove

Debido a que `find_all()` es el método más popular en la API de búsqueda de Beautiful Soup, puedes usar un acceso directo para llamarlo. Si tratas el objeto BeautifulSoup como si fuera una función, es lo mismo que llamar a `find_all()` en ese objeto.

Estas dos líneas de código son equivalentes:

In [12]:
a_tags = soup.find_all("a")
a_tags_alt = soup("a")
print(a_tags[0])
print(a_tags_alt[0])

<a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a>
<a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a>


¿Cuántos enlaces obtuvimos?

In [13]:
print(len(a_tags))

213


¡Eso es demasiado! Muchos elementos en una página tendrán la misma etiqueta HTML. Por ejemplo, si buscas todos los elementos con la etiqueta `a`, es probable que obtengas más resultados, muchos de los cuales tal vez no quieras. Recuerda, la etiqueta `a` define un hipervínculo, por lo que generalmente encontrarás muchos en cualquier página.

¿Qué pasa si queremos buscar etiquetas HTML con atributos específicos, como clases CSS particulares?

Podemos hacer esto agregando un argumento adicional a `find_all`. En el siguiente ejemplo, estamos buscando todas las etiquetas `a` y luego filtrando aquellas con `class_="sidemenu"`.

In [ ]:
# Obtener solo las etiquetas 'a' en la clase 'sidemenu'
side_menus = soup("a", class_="sidemenu")
side_menus[:5]

[<a class="sidemenu" href="/senate/default.asp">  Members  </a>,
 <a class="sidemenu" href="/senate/committees/default.asp">  Committees  </a>,
 <a class="sidemenu" href="/senate/schedules/default.asp">  Schedules  </a>,
 <a class="sidemenu" href="/senate/journals/default.asp">  Journals  </a>,
 <a class="sidemenu" href="/senate/transcripts/default.asp">  Transcripts  </a>]

Una forma más eficiente de buscar elementos en un sitio web es mediante un **selector CSS**. Para esto, usamos un método diferente llamado `.select()`. Solo tienes que pasar una cadena de texto al `.select()` para obtener todos los elementos que coincidan con esa cadena como un selector CSS válido.

En el ejemplo anterior, podemos usar `"a.sidemenu"` como un selector CSS, lo que devuelve todas las etiquetas a con la clase `"a.sidemenu"` .


In [ ]:
# Obtener elementos con el selector CSS "a.sidemenu"
selected = soup.select("a.sidemenu")
selected[:5]

[<a class="sidemenu" href="/senate/default.asp">  Members  </a>,
 <a class="sidemenu" href="/senate/committees/default.asp">  Committees  </a>,
 <a class="sidemenu" href="/senate/schedules/default.asp">  Schedules  </a>,
 <a class="sidemenu" href="/senate/journals/default.asp">  Journals  </a>,
 <a class="sidemenu" href="/senate/transcripts/default.asp">  Transcripts  </a>]

## 🥊 Reto: Encontrar Todos

Usa BeautifulSoup para encontrar todos los elementos `a` con la clase `mainmenu`.

Aquí tenemos un ejemplo de cómo hacerlo:

In [ ]:
# Aqui tenemos el codigo
main_menus = soup.select("a.mainmenu")
main_menus[:5]


[<a class="mainmenu" href="/">Home</a>,
 <a class="mainmenu" href="/legislation/" onblur="HM_f_PopDown('elMenu1')" onfocus="HM_f_PopUp('elMenu1',event)" onmouseout="HM_f_PopDown('elMenu1')" onmouseover="HM_f_PopUp('elMenu1',event)">Legislation &amp; Laws</a>,
 <a class="mainmenu" href="/senate/" onblur="HM_f_PopDown('elMenu3')" onfocus="HM_f_PopUp('elMenu3',event)" onmouseout="HM_f_PopDown('elMenu3')" onmouseover="HM_f_PopUp('elMenu3',event)">Senate</a>,
 <a class="mainmenu" href="/house/" onblur="HM_f_PopDown('elMenu2')" onfocus="HM_f_PopUp('elMenu2',event)" onmouseout="HM_f_PopDown('elMenu2')" onmouseover="HM_f_PopUp('elMenu2',event)">House</a>,
 <a class="mainmenu" href="/mylegislation/" onblur="HM_f_PopDown('elMenu4')" onfocus="HM_f_PopUp('elMenu4',event)" onmouseout="HM_f_PopDown('elMenu4')" onmouseover="HM_f_PopUp('elMenu4',event)">My Legislation</a>]

## Paso 4: Obtener Atributos y Texto de los Elementos

Una vez que identificamos los elementos, queremos acceder a la información dentro de ese elemento. Generalmente, esto significa dos cosas:
1. Texto
2. Atributos

Obtener el texto dentro de un elemento es sencillo. Solo tenemos que usar el atributo `text` de un objeto `tag`. 

Aquí tienes un ejemplo:


In [ ]:
# Obtener todos los enlaces con la clase 'sidemenu'
side_menu_links = soup.select("a.sidemenu")

# Examinar el primer enlace
first_link = side_menu_links[0]
print(first_link)

# ¿Qué clase tiene esta variable?
print('Class: ', type(first_link))

<a class="sidemenu" href="/senate/default.asp">  Members  </a>
Class:  <class 'bs4.element.Tag'>


¡Es una etiqueta de Beautiful Soup! Esto significa que tiene un miembro de `text`:

In [18]:
print(first_link.text)

  Members  


A veces queremos el valor de ciertos atributos. Esto es particularmente relevante para las etiquetas `a`, o enlaces, donde el atributo `href` nos dice a dónde lleva el enlace.

💡 **Consejo**: Puedes acceder a los atributos de una etiqueta tratándola como un diccionario.

In [19]:
print(first_link['href'])

/senate/default.asp


## 🥊 Reto: Extraer atributos específicos
Para extraer todos los atributos `href` de las URLs de los enlaces con la clase `mainmenu`, podemos usar el siguiente código:

In [ ]:
# AQUI TENEMOS EL CODIGO
for link in main_menus:
    print(link['href'])

for i in range(2):
    print(main_menus[i]['href'])

/
/legislation/
/senate/
/house/
/mylegislation/
/sitemap.asp
/
/legislation/


<a id='scrape'></a>

# Extraer Información de la Asamblea General de Illinois

Creas o no, esas son realmente las herramientas fundamentales que necesitas para extraer informacion de un sitio web. Una vez que pases más tiempo familiarizándote con HTML y CSS, entonces simplemente es cuestión de entender la estructura de un sitio web en particular y aplicar inteligentemente las herramientas de Beautiful Soup y Python.

Vamos a aplicar estas habilidades para extaer la [98ª Asamblea General de Illinois](http://www.ilga.gov/senate/default.asp?GA=98).

Específicamente, nuestro objetivo es extraer información sobre cada senador, incluyendo su nombre, distrito y partido.

## Extraer y Analizar la Página Web

Vamos a extraer y analizar la página web, utilizando las herramientas que aprendimos en la sección anterior.

In [ ]:
# Realizar una solicitud GET
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Leer el contenido de la respuesta del servidor
src = req.text
# Procesar con Beautiful Soup
soup = BeautifulSoup(src, "lxml")

## Buscar los elementos de la tabla

Nuestro objetivo es obtener los elementos de la tabla en la página web. Recuerda: las filas se identifican con la etiqueta `tr`. Vamos a usar `find_all` para obtener estos elementos.

In [ ]:
# Obtener todos los elementos de las filas de la tabla
rows = soup.find_all("tr")
len(rows)

73

⚠️ **Advertencia**: Ten en cuenta: `find_all` obtiene *todos* los elementos con la etiqueta `tr`. Nosotros solo queremos algunos de ellos. 

Si usamos la función "Inspeccionar" en Google Chrome y miramos cuidadosamente, podemos usar algunos selectores CSS para obtener solo las filas que nos interesan. Específicamente, queremos las filas internas de la tabla:

In [ ]:
# Devuelve todos los selectores CSS 'tr tr tr' en la página
rows = soup.select('tr tr tr')

for row in rows[:5]:
    print(row, '\n')

<tr><td colspan="5">
<span class="heading">Illinois State Senators</span>
<span class="italics">  98th  General Assembly</span><br/>
<!-- 3/2/09 temp comment out until fixed for GA specific-->
<!-- add 97th ga currently no info -->
<a href="98GA_Senate_Leadership.pdf">Leadership</a> <a href="98th_Senate_Officers.pdf">Officers</a> <a href="98GA_Senate_Seating_Chart.pdf">Senate Seating Chart</a>  <span class="content"><b>Democrats:</b> 40   <b>Republicans:</b> 19</span><br/>
</td></tr> 

<tr>
<td class="header" width="45%"><a class="filetab" href="javascript:Sort('LastName','',98);" title="Sort by Senator">Senator</a></td>
<td align="center" class="header" width="15%">Bills</td>
<td align="center" class="header" width="10%">Committees</td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript:Sort('DistrictNumber','',98);" title="Sort by District">District</a></td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript:Sort('Party','

Parece que queremos todo después de las primeras dos filas. Comencemos trabajando con una sola fila y construyamos nuestro bucle a partir de ahí.

In [24]:
example_row = rows[2]
print(example_row.prettify())

<tr>
 <td bgcolor="white" class="detail" width="40%">
  <a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">
   Pamela J. Althoff
  </a>
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  <a href="SenatorBills.asp?GA=98&amp;MemberID=1911">
   Bills
  </a>
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  <a href="SenCommittees.asp?GA=98&amp;MemberID=1911">
   Committees
  </a>
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  32
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  R
 </td>
</tr>



Desglosamos esta fila en sus celdas/columnas utilizando el método `select` con selectores CSS. Al observar cuidadosamente el HTML, hay un par de formas en las que podríamos hacerlo.

* Podríamos identificar las celdas por su etiqueta `td`.
* Podríamos usar el nombre de la clase `.detail`.
* Podríamos combinar ambos y usar el selector `td.detail`.

In [25]:
for cell in example_row.select('td'):
    print(cell)
print()

for cell in example_row.select('.detail'):
    print(cell)
print()

for cell in example_row.select('td.detail'):
    print(cell)
print()

<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td>
<td align="center" bgcolor="white" class="detail" width="15%">32</td>
<td align="center" bgcolor="white" class="detail" width="15%">R</td>

<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td>
<td align="center" bgcolor="white" class

Podemos confirmar que todos estos son iguales.

In [26]:
assert example_row.select('td') == example_row.select('.detail') == example_row.select('td.detail')

Usamos el selector `td.detail` para ser lo más específicos posibles.

In [ ]:
# Selecciona solo aquellas etiquetas 'td' con la clase 'detail'
detail_cells = example_row.select('td.detail')
detail_cells

[<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%">32</td>,
 <td align="center" bgcolor="white" class="detail" width="15%">R</td>]

La mayoría de las veces, nos interesa el **texto** real de un sitio web, no sus etiquetas. Recuerda que para obtener el texto de un elemento HTML, usamos el miembro `text`:

In [ ]:
# Mantenemos solo el texto en cada una de esas celdas
row_data = [cell.text for cell in detail_cells]

print(row_data)

['Pamela J. Althoff', 'Bills', 'Committees', '32', 'R']


¡Se ve bien! Ahora solo usamos nuestros conocimientos básicos de Python para obtener los elementos de esta lista que necesitamos. 

Recuerda, queremos el nombre del senador, su distrito y su partido.

In [29]:
print(row_data[0]) # Name
print(row_data[3]) # District
print(row_data[4]) # Party

Pamela J. Althoff
32
R


## Desaciéndonos de las filas basura

Vimos que en el principio que no todas las files que se obtienen corresponden a un Senador. Vamos a necesitar hacer un poco de limpieza antes de seguir. Veamos algunos ejemplos:

In [30]:
print('Fila 0:\n', rows[0], '\n')
print('Fila 1:\n', rows[1], '\n')
print('Última fila:\n', rows[-1])

Fila 0:
 <tr><td colspan="5">
<span class="heading">Illinois State Senators</span>
<span class="italics">  98th  General Assembly</span><br/>
<!-- 3/2/09 temp comment out until fixed for GA specific-->
<!-- add 97th ga currently no info -->
<a href="98GA_Senate_Leadership.pdf">Leadership</a> <a href="98th_Senate_Officers.pdf">Officers</a> <a href="98GA_Senate_Seating_Chart.pdf">Senate Seating Chart</a>  <span class="content"><b>Democrats:</b> 40   <b>Republicans:</b> 19</span><br/>
</td></tr> 

Fila 1:
 <tr>
<td class="header" width="45%"><a class="filetab" href="javascript:Sort('LastName','',98);" title="Sort by Senator">Senator</a></td>
<td align="center" class="header" width="15%">Bills</td>
<td align="center" class="header" width="10%">Committees</td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript:Sort('DistrictNumber','',98);" title="Sort by District">District</a></td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascr

Cuando utilizamos el bucle "for", nosotros solo queremos aplicarlo en las filas más relevantes. Por lo que necesitamos filtrar las filas irrelevantes. La manera de hacer esto es comparando algunas de las filas que queremos, para ver que las hace diferentes, y luego utilizar esa información en un condicional.

Como lo puedes imaginar, hay muchas maneras de hacer esto, y va a depender del sitio web. Vamos a mostrar algunas maneras aquí para darte una idea de como hacerlo.

In [ ]:
# Filas malas
print(len(rows[0]))
print(len(rows[1]))

# Filas buenas
print(len(rows[2]))
print(len(rows[3]))

1
11
5
5


Tal vez las filas buenas tienen una longitud de 5. Revisemos:

In [ ]:
good_rows = [row for row in rows if len(row) == 5]

# Comprobemos algunas filas
print(good_rows[0], '\n')
print(good_rows[-2], '\n')
print(good_rows[-1])

<tr><td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td><td align="center" bgcolor="white" class="detail" width="15%">32</td><td align="center" bgcolor="white" class="detail" width="15%">R</td></tr> 

<tr><td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=2035">Patricia Van Pelt</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=2035">Bills</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=2035">Committees</a></td><td align="center" bgcolor="white

Encontramos una fila footer en nuestra lista que nos gustaría evitar. Vamos a intentar otra cosa:

In [33]:
rows[2].select('td.detail') 

[<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%">32</td>,
 <td align="center" bgcolor="white" class="detail" width="15%">R</td>]

In [ ]:
# Fila mala
print(rows[-1].select('td.detail'), '\n')

# Fila buena
print(rows[5].select('td.detail'), '\n')

# ¿Qué tal esto?
good_rows = [row for row in rows if row.select('td.detail')]

print("Checking rows...\n")
print(good_rows[0], '\n')
print(good_rows[-1])

[] 

[<td bgcolor="EBEBEB" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=2022">Jennifer Bertino-Tarrant</a></td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=2022">Bills</a></td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=2022">Committees</a></td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%">49</td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%">D</td>] 

Checking rows...

<tr><td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></

Parece que encontramos algo que ha funcionado!

## Consolidarlo todo

Ahora que hemos visto como obtener los datos que queremos de una fila, así como también filtrar las filas que no queremos, vamos a consolidar todo en un bucle.

In [ ]:
# Definir lista de almacenamiento
members = []

# Eliminar filas innecesarias
valid_rows = [row for row in rows if row.select('td.detail')]

# Iterar a través de todas las filas
for row in valid_rows:
    # Seleccionar solo aquellas etiquetas 'td' con clase 'detail'
    detail_cells = row.select('td.detail')
    # Mantener solo el texto en cada una de esas celdas
    row_data = [cell.text for cell in detail_cells]
    # Recopilar la información
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]
    # Almacenar en una tupla
    senator = (name, district, party)
    # Agregar a la lista
    members.append(senator)

In [ ]:
# Debe ser 61
len(members)

61

Vamos a ver que tenemos en `members`.

In [37]:
print(members[:5])

[('Pamela J. Althoff', 32, 'R'), ('Jason A. Barickman', 53, 'R'), ('Scott M Bennett', 52, 'D'), ('Jennifer Bertino-Tarrant', 49, 'D'), ('Daniel Biss', 9, 'D')]


## 🥊  Reto: Obtener elementos `href` que apunten a los proyectos de ley de los miembros  

El código anterior recupera información sobre:  

- el nombre del senador,  
- su número de distrito,  
- y su partido.  

Ahora queremos obtener la URL de la lista de proyectos de ley de cada senador. Cada URL seguirá un formato específico.  

El formato para la lista de proyectos de ley de un senador dado es:  

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`  

para obtener algo como:  

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`  

donde `MEMBER_ID=1911`.  

Deberías notar que, desafortunadamente, `MEMBER_ID` no se está extrayendo actualmente en nuestro código de scraping.  

Tu tarea inicial es modificar el código anterior para que también **recupere la URL completa que apunta a la página correspondiente de proyectos de ley patrocinados principalmente**, para cada miembro, y la devuelva junto con su nombre, distrito y partido.  

### Consejos:  

* Para hacer esto, querrás obtener el elemento de anclaje (`<a>`) apropiado en la fila de la tabla de cada legislador. Puedes usar nuevamente el método `.select()` en el objeto `row` dentro del bucle para hacer esto, de manera similar al comando que encuentra todas las celdas `td.detail` en la fila. Recuerda que solo queremos el enlace a los proyectos de ley del legislador, no los comités ni la página de perfil del legislador.  
* Los elementos de anclaje en el HTML se verán como `<a href="/senate/Senator.asp/...">Bills</a>`. La cadena en el atributo `href` contiene el **enlace relativo** que estamos buscando. Puedes acceder a un atributo de un objeto `Tag` de BeautifulSoup de la misma manera que accedes a un diccionario en Python: `anchor['attributeName']`. Consulta la <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentación</a> para más detalles.  
* Hay _muchas_ formas diferentes de usar BeautifulSoup para hacer esto. Cualquier método que utilices para extraer el `href` está bien.  

El código ha sido parcialmente completado para ti. Complétalo donde dice `#YOUR CODE HERE`. Guarda la ruta en un objeto llamado `full_path`.  


In [ ]:
#  Hacer una solicitud GET
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Leer el contenido de la respuesta del servidor
src = req.text
# Parsear el contenido
soup = BeautifulSoup(src, "lxml")
# Crear una lista vacía para almacenar nuestros datos
members = []

# Devuelve cada selector ‘tr tr tr’ CSS en la página
rows = soup.select('tr tr tr')
# Eliminar las filas innecesarias
rows = [row for row in rows if row.select('td.detail')]

# Iterar sobre todas las filas
for row in rows:
    # Seleccionar solo aquellas etiquetas 'td' con la clase 'detail'
    detail_cells = row.select('td.detail') 
    # KMantener solo el texto en cada una de esas celdas
    row_data = [cell.text for cell in detail_cells]
    #  Recoger información
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]

    # TU CÓDIGO AQUÍ
    #full_path = ''
    # Buscar el enlace a los proyectos de ley dentro de la fila
    bill_anchor = row.select_one('td.detail a[href*="SenatorBills.asp"]')
    
    # Extraer el 'href' si existe, y construir la URL completa
    if bill_anchor:
        relative_path = bill_anchor['href']
        full_path = f"http://www.ilga.gov{relative_path}"
    else:
        full_path = ''

    #  Almacenar en una tupla
    senator = (name, district, party, full_path)
    #  Añadir a la lista
    members.append(senator)

In [ ]:
# Descomentar para probar 
members[:5]

[('Pamela J. Althoff',
  32,
  'R',
  'http://www.ilga.govSenatorBills.asp?GA=98&MemberID=1911'),
 ('Jason A. Barickman',
  53,
  'R',
  'http://www.ilga.govSenatorBills.asp?GA=98&MemberID=2018'),
 ('Scott M Bennett',
  52,
  'D',
  'http://www.ilga.govSenatorBills.asp?GA=98&MemberID=2272'),
 ('Jennifer Bertino-Tarrant',
  49,
  'D',
  'http://www.ilga.govSenatorBills.asp?GA=98&MemberID=2022'),
 ('Daniel Biss',
  9,
  'D',
  'http://www.ilga.govSenatorBills.asp?GA=98&MemberID=2020')]

## 🥊  Reto: Modulariza tu código  

Convierte el código anterior en una función que acepte una URL, extraiga los senadores de esa URL y devuelva una lista de tuplas que contengan información sobre cada senador.  

In [ ]:
# AQUI ESTA EL CODIGO
def get_members(url):
    # Hacer la solicitud GET
    req = requests.get(url)
    # Leer el contenido de la respuesta del servidor
    src = req.text
    # Analizar con BeautifulSoup
    soup = BeautifulSoup(src, "lxml")

    # Crear una lista vacía para almacenar los datos
    members = []

    # Obtener todas las filas 'tr tr tr' que contienen datos
    rows = soup.select('tr tr tr')
    # Filtrar las filas basura
    rows = [row for row in rows if row.select('td.detail')]

    # Iterar sobre cada fila
    for row in rows:
        # Seleccionar solo las celdas 'td' con clase 'detail'
        detail_cells = row.select('td.detail')  
        # Extraer el texto de cada celda
        row_data = [cell.text.strip() for cell in detail_cells]

        # Extraer información relevante
        name = row_data[0]
        district = int(row_data[3])
        party = row_data[4]

        # Buscar el enlace a los proyectos de ley dentro de la fila
        bill_anchor = row.select_one('td.detail a[href*="SenatorBills.asp"]')

        # Extraer el 'href' si existe, y construir la URL completa
        if bill_anchor:
            relative_path = bill_anchor['href']
            
            # Verificar si la URL ya tiene el dominio base
            if relative_path.startswith('http://www.ilga.gov'):
                full_path = relative_path  # Si ya tiene la URL completa, usarla tal cual
            elif relative_path.startswith('/'):
                full_path = f"http://www.ilga.gov{relative_path}"  # Concatenar solo si es relativa
            else:
                # Si no tiene la barra inicial, corregirla
                full_path = f"http://www.ilga.gov/{relative_path}"
        else:
            full_path = ''


        # Almacenar en una tupla
        senator = (name, district, party, full_path)
        # Agregar a la lista
        members.append(senator)

    return members


In [ ]:
# Prueba tu código
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = get_members(url)
len(senate_members)

61

## 🥊 Reto para llevar a casa: Escribir una función de scraping  

Queremos extraer información de las páginas web correspondientes a los proyectos de ley patrocinados por cada legislador.  

Escribe una función llamada `get_bills(url)` para analizar una URL de proyectos de ley dada. Esto implicará:  

- hacer una solicitud a la URL usando la librería <a href="http://docs.python-requests.org/en/latest/">`requests`</a>  
- usar las funciones de la librería `BeautifulSoup` para encontrar todos los elementos `<td>` con la clase `billlist`  
- devolver una _lista_ de tuplas, cada una con:  
    - la descripción (2ª columna)  
    - la cámara (S o H) (3ª columna)  
    - la última acción (4ª columna)  
    - la fecha de la última acción (5ª columna)  

Esta función ha sido parcialmente completada. Complétala.  


In [ ]:
def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src)
    rows = soup.select('tr')
    bills = []
    for row in rows:
        # AQUI ESTA EL CODIGO
        cells = row.select('td.billlist')

        # Verificar que la fila tenga suficientes columnas
        if len(cells) >= 5:
            bill_id = cells[0].text.strip()  # ID del proyecto de ley
            description = cells[1].text.strip()  # Descripción
            chamber = cells[2].text.strip()  # Cámara (S o H)
            last_action = cells[3].text.strip()  # Última acción
            last_action_date = cells[4].text.strip()  # Fecha de la última acción
            bill = (bill_id, description, chamber, last_action, last_action_date)
            bills.append(bill)
    return bills

In [ ]:
# "Descomenta para probar tu código"
test_url = senate_members[0][3]

print("Raw test_url:", test_url)

# Verificar si test_url comienza con la base correcta
base_url = "http://www.ilga.gov"
if test_url.startswith(base_url):
    # Si test_url comienza con la URL base, asegurarse de agregar la palabra 'senate/' entre la base y la ruta
    test_url = test_url[:len(base_url)] + "/senate/" + test_url[len(base_url):]
else:
    # Si no empieza con "http", corregirla añadiendo la base URL correctamente
    if not test_url.startswith('/'):
        test_url = "/" + test_url  # Asegurarse de que comience con una barra '/'
    
    # Concatenar correctamente la URL
    test_url = base_url + "/senate/" + test_url  # Concatenar la URL base con la parte relativa

# Imprimir la URL corregida para verificar que esté bien
print("Fixed test_url:", test_url)  # Verifica que ahora sea válida

get_bills(test_url)[0:5]

Raw test_url: http://www.ilga.gov/SenatorBills.asp?GA=98&MemberID=1911
Fixed test_url: http://www.ilga.gov/senate//SenatorBills.asp?GA=98&MemberID=1911


[('SB2', 'STATE GOVERNMENT-TECH', 'S', 'Session Sine Die', '1/13/2015'),
 ('SB2', 'STATE GOVERNMENT-TECH', 'S', 'Session Sine Die', '1/13/2015'),
 ('SB2', 'STATE GOVERNMENT-TECH', 'S', 'Session Sine Die', '1/13/2015'),
 ('SB9',
  'PUBLIC UTIL-PERFORMANCE-BASED',
  'S',
  'Public Act . . . . . . . . . 98-0015',
  '5/23/2013'),
 ('SB27', 'MEDICAID BUDGET NOTE ACT', 'S', 'Session Sine Die', '1/13/2015')]

### Raspado de Todos los Proyectos de Ley

Finalmente, crea un diccionario `bills_dict` que asocie un número de distrito (la clave) con una lista de proyectos de ley (el valor) provenientes de ese distrito. Puedes hacerlo iterando sobre todos los miembros del senado en `members_dict` y llamando a `get_bills()` para cada una de sus URLs asociadas de proyectos de ley.

**NOTA:** por favor, llama a la función `time.sleep(1)` en cada iteración del bucle, para no sobrecargar el sitio web del estado.

In [91]:
bills_dict = {}

# Iterar sobre todos los senadores en la lista 'members'
for senator in members:
    # Obtener el número de distrito del senador (índice 1 de la tupla)
    district = senator[1]
    
    # Obtener la URL de los proyectos de ley de este senador (índice 3 de la tupla)
    bills_url = senator[3]
    
    # Depuración: Verificar la URL original solo una vez
    if not bills_url.startswith("http"):
        print(f"Raw bills URL: {bills_url}")
    
    # Corregir la URL si no tiene el formato correcto
    if bills_url and not bills_url.startswith("http"):
        # Si la URL es relativa, agregar la base correcta
        if bills_url.startswith("/"):
            bills_url = "http://www.ilga.gov" + bills_url
        else:
            # Si la URL no tiene '/', agregamos el dominio con una barra
            bills_url = "http://www.ilga.gov/" + bills_url
    
    # Asegurarse de que la palabra "senate" esté en la URL antes de 'SenatorBills.asp'
    if "SenatorBills.asp" in bills_url:
        # Agregar 'senate' (minúsculas) a la URL justo antes de 'SenatorBills.asp'
        bills_url = bills_url.replace("SenatorBills.asp", "senate/SenatorBills.asp")
    
    # Corregir la URL para evitar concatenación incorrecta
    if bills_url.startswith("http://www.ilga.gov") and not bills_url.startswith("http://www.ilga.gov/"):
        # Si ya tiene el dominio, asegurarnos de que la URL contenga la barra '/'
        bills_url = "http://www.ilga.gov/" + bills_url[len("http://www.ilga.gov"):]
    
    # Depuración: Verificar la URL corregida solo una vez
    #print(f"Fixed bills URL: {bills_url}")
    
    # Llamar a la función get_bills() para obtener los proyectos de ley
    bills = get_bills(bills_url)
    
    # Agregar los proyectos de ley al diccionario bills_dict
    if district not in bills_dict:
        bills_dict[district] = []  # Si el distrito no está en el diccionario, agregarlo
    
    bills_dict[district].extend(bills)  # Agregar los proyectos de ley a la lista correspondiente al distrito
    
    # Pausar la ejecución para evitar sobrecargar el servidor
    time.sleep(1)

In [ ]:
# Descomenta para probar tu código
bills_dict[52]

[('SR1730',
  'MEMORIAL - THOMAS G. HAYS',
  'S',
  'Resolution Adopted',
  '1/13/2015'),
 ('SR1730',
  'MEMORIAL - THOMAS G. HAYS',
  'S',
  'Resolution Adopted',
  '1/13/2015'),
 ('SR1730',
  'MEMORIAL - THOMAS G. HAYS',
  'S',
  'Resolution Adopted',
  '1/13/2015'),
 ('SB10',
  'CIVIL LAW-TECH',
  'S',
  'Public Act . . . . . . . . . 98-0597',
  '11/20/2013'),
 ('SB10',
  'CIVIL LAW-TECH',
  'S',
  'Public Act . . . . . . . . . 98-0597',
  '11/20/2013'),
 ('SB10',
  'CIVIL LAW-TECH',
  'S',
  'Public Act . . . . . . . . . 98-0597',
  '11/20/2013'),
 ('SB16', 'EDUCATION-TECH', 'S', 'Session Sine Die', '1/13/2015'),
 ('SB30',
  'THOMSON PRISON CESSION ACT',
  'S',
  'Public Act . . . . . . . . . 98-0070',
  '7/15/2013'),
 ('SB34', 'HLTH BENEFITS EX-ADMIN', 'S', 'Session Sine Die', '1/13/2015'),
 ('SB46',
  'UNIVERSITY OF ILLINOIS TRUSTEE',
  'S',
  'Session Sine Die',
  '1/13/2015'),
 ('SB47',
  'PUB AID-CAUSE OF ACTION-NOTICE',
  'S',
  'Public Act . . . . . . . . . 98-0073',
  '7/15